# Gaussian Processes and Hyperparameter optimization with Bayesian optimization

In this lab, we aim to get a better understanding of Gaussian Processes (GPs) and Bayesian Optimzation (BO). 

GPs are regression models with well calibrated uncertainty estimates and Bayesian optimization (BO) is a tool to optimize black-box function, i.e., functions with unknown structure that are expensive to evaluate.
BO consists of the following steps that are repeated until one runs out of time or money:

1. Learn a model of the function
2. Using this model, find a promising point to evaluate next
3. Evaluate the point
4. Refine the model using the new observation

The lab consists of two parts. In **PART 1**, we will write our own implementation of a Gaussian Process to understand its differents parts. We will also use it for some simple BO. In **PART 2**, we will try to apply state-of-the-art BO tools to optimize the hyperparameters of a Artifical Neural Network. We will use gpytorch for GPs and botorch for BO.

***
## PART 1
***
In this part we will implement our own GP and test it in a simple BO setting.


_Tips_:
This lab will require you to work with a lot of matrices both in numpy and in torch. You can use `@` to do matrix multiplication. Also, when working with matrices, keeping track of matrix dimensions is essential. Make sure to keep an eye on it.

In [ ]:
from typing import Callable                  # typing is a package that provides special type-hints to 
                                             # function defintions. 'Callable' is a type hint for a function.
                                             # That is, if a function func(c: Callable):... takes a Callable c
                                             # as input, then c should be a function itself.

import numpy as np
import scipy as sp                           # Scipy is an optimization library
import seaborn as sns                        # Seaborn is a package for making nicer plots
from matplotlib import pyplot as plt

### We first define a synthetic benchmark function we will work on

In [ ]:
def fn(
        x: np.ndarray
) -> np.ndarray:
    y = np.sin(x) + np.sin((10.0 / 4.0) * x)
    return y


# define lower and upper bounds of the function: we will only consider the function within these bounds
lb, ub = -2.7, 7.5

plt.plot(np.linspace(lb, ub, 200), fn(np.linspace(lb, ub, 200)), label='f(x)')
plt.legend()
plt.show()

### Defining the training and test data
Our goal is to eventually train a GP regressor, and for this we will need to have a training data set. The function in the plot above is the true function we are trying to predict, and the red dots in the plot below are the data points that we will train our model on.

In [ ]:
x_train = np.random.RandomState(1).rand(5) * (ub - lb) + lb
x_train = x_train[:, np.newaxis]
y_train = fn(x_train)

x_test = np.linspace(lb, ub, 250)[:, np.newaxis]
y_test = fn(x_test)

plt.plot(np.linspace(lb, ub, 200), fn(np.linspace(lb, ub, 200)), label='f(x)')
plt.scatter(x_train, y_train, color="red", zorder=10)

# Implementing a Gaussian process

One view of GPs is as the assumption that any set of y-values ($y_1, y_2,..,y_n$) follows a joint normal distribution with some mean $\mu$ and covariance matrix $K$, where the elements of $K$ are defined by the kernel function $K_{ij} = k(x_i, x_j)$. 

The mean function describes the average value of a point and the covariance function essentially captures how much two points are correlated.

In our example, we will $\mu=0$ use the RBF kernel:

$$
 k(\mathbf{x},\mathbf{x'}) = \exp\left(-\frac{||\mathbf{x}-\mathbf{x}'||^2}{2l^2}\right).
$$

This kernel-function grows larger the closer $\mathbf{x}$ and $\mathbf{x'}$ are to each other, i.e., it assumes that the relation of two points is only determined by their distance. Such kernels are called stationary. There are other kernels whose value depends on the definite location of the points, though we will not see them in this lab. The distance is scaled by the parameter $l$ and we will see later how it affects the GP as well as how it can be learned from the data.

Under the assumption that 

$$
(y_1, y_2, ..,y_n) \sim \mathcal{GP}(\mu, K),
$$

we can then make predictions for new points through the standard rules for conditioning for multivariate Gaussian distributions. If we have training data $(y_1, y_2, ..,y_n)$ and want to predict a new point $y^*$ for features $x^*$, then with

$$
K = \begin{bmatrix}
K & k^* \\
k^* & k^{**}
\end{bmatrix},
$$
$$
k^* = (k(x_1, x^*), k(x_2, x^*), .., k(x_n, x^*),
$$
$$
\text{and}~~ k^{**} = k(x^*, x^*)
$$
we have that 
$$
y^*\sim \mathcal{N}(k^*K^{-1}y, k^{**} - k^*K^{-1}k^{*T})
$$

---
**Bonus info:**

A GP can be seen as a probability distribution over _functions_. Hence, we can sample functions f from a GP and we write that they are distributed according to:

$$
f(\mathbf{x}) \sim \mathcal{GP}(\mu(\mathbf{x}), k(\mathbf{x},\mathbf{x'})).
$$




## Defining an RBF kernel with a global lengthscale

The kernel function decides how "connected" two data points $x_1$ and $x_2$ are.

**Your task:** Implement the RBF kernel as described above.

**Question:** What does a return value of 1 mean, what does a return value of 0 mean?

In [ ]:
def kern(X: np.ndarray, X_prime: np.ndarray, ls: float) -> np.ndarray:
    """
    Returns the covariance matrix between X and X'. 
    
    NOTE: This function should handle calculating the kernel function between several data points at once.
    Both X and X' can be considered matrices with dimensions (nxd) and (mxd), respectively. The output
    should hence be a matrix of shape (nxm) and contain the value of the kernel function between every pair between X and X'. 
    
    If X or X' are 1d vectors (with d elements), we first reshape them to (1,d)-matrices.
    """

    if X.ndim == 1:
        X = X.reshape(1, -1)
    if X_prime.ndim == 1:
        X_prime = X_prime.reshape(1, -1)
        
    # ➡️ TODO : implement the RBF kernel ⬅️
    
    return ...

#### Safety check

The cell below should display the following image:

![RBF kernel plot](kernel_plot.png)

**Question:** What does this image show?

In [ ]:
GRANULARITY = 50

x = np.linspace(-2, 2, GRANULARITY).reshape(-1, 1)
y = np.linspace(-2, 2, GRANULARITY).reshape(-1, 1)

xlabels = [''] * GRANULARITY
ylabels = [''] * GRANULARITY

for idx in np.arange(GRANULARITY)[::5]:
    xlabels[idx] = f"{x.squeeze()[idx]:.2f}"
    ylabels[idx] = f"{y.squeeze()[idx]:.2f}"

z = kern(x, y, 0.5)
assert z.ndim == 2, "kern must be a two dimensional matrix"
assert z.shape == (x.shape[0], x.shape[0]), f"kern should in this case yield a {x.shape[0]} by {x.shape[0]} matrix"
sns.heatmap(z, xticklabels=xlabels, yticklabels=ylabels)
plt.show()

## Defining a mean function (constant zero)

Surprisingly, the mean function is not really important. 
All we are interested in is the posterior of the GP, i.e., the distribution after we have seen some data.
In the end, it turns out that the prior mean (which is the mean function here) is unimportant for that.
Therefore, we can just implement the mean function as constant zero. This is especially true, as we tend to standardize our data so that is is zero mean.

**Your task:** Implement the mean function.



In [ ]:
def mean(x: np.ndarray) -> np.ndarray:
    """
    Returns the mean. 

    Note: The mean should be a two-dimensional column vector - one 0 per data point. 
    
    If x is a 1d vector with d elements, reshape it to a (1 x d) matrix first.
    Should return a 2d zeros vector.
    
    """

    # ➡️ TODO : reshape to 2d ⬅️
    
    # ➡️ TODO : implement the mean function ⬅️

    return ...

#### Safety check

The output of the following cell should be __exactly__ `array([[0.], [0.], [0.]]) (1, 1)`

In [ ]:
print(mean(np.zeros((3, 4))), mean(np.zeros(4)).shape)

## Simple GP implementation

We will now use the mean and covariance/kernel functions to build a GP. 
In the `__call__` function, we will compute the posterior distribution for a vector of points `x`. The output will __not__ be an array, but a __multivariate Gaussian__ with the dimensionality of the length of `x`.

As mentioned above, but here with slightly more detailed notation, the posterior mean $\mu_n$ of a GP is calculated as follows:
$$
\mu_n(\mathbf{x^*}) = k(\mathbf{x^*},\mathbf{x}_{1:n})k(\mathbf{x}_{1:n},\mathbf{x}_{1:n})^{-1}(y(\mathbf{x}_{1:n})-\mu(\mathbf{x}_{1:n}))+\mu(\mathbf{x^*}).
$$
 $\mathbf{x}_{1:n}$ is the training data for which we have observed function values and $\mathbf{x^*}$ is the point for which we want to make predictions.

The posterior variance is calculated as 
$$
\sigma_n(\mathbf{x}) = k(\mathbf{x^*},\mathbf{x^*})-k(\mathbf{x^*},\mathbf{x}_{1:n})k(\mathbf{x}_{1:n},\mathbf{x}_{1:n})^{-1}k(\mathbf{x^*},\mathbf{x}_{1:n})^{\intercal}.
$$

In the `fit`  function, we will set the training data and calculate a critical component: the inverse of the covariance matrix $k(\mathbf{x}_{1:n},\mathbf{x}_{1:n})^{-1}$.
Usually, this is done using Cholesky decomposition but we will use `np.linalg.inv` to calculate the inverse.

Pro tip: You might run into problems when computing the inverse due to numerical instabilities. If this happens, add a value of $10^{-6}$ to the diagonal elements of the matrix before computing the inverse.

**Your task:** Compute the covariance matrix $k(\mathbf{x}_{1:n},\mathbf{x}_{1:n})$ and its inverse in `initialize` and calculate the posterior distribution in `__call__`.

In [ ]:
class GaussianProcess:
    def __init__(
            self,
            mean_function: Callable[[np.ndarray], np.ndarray],
            kern_function: Callable[[np.ndarray, np.ndarray], np.ndarray],
            ls: float
    ):
        # setting the mean function
        self.mean = mean_function
        # setting the kernel function, fix the lengthscale so we don't have to pass it all the time
        self.kern = lambda x, y: kern_function(x, y, ls)
        self.x_train = None
        self.y_train = None

    def initialize(self, x_train: np.ndarray, y_train: np.ndarray) -> 'GaussianProcess':
        # some checks
        assert x_train.ndim == 2 and y_train.ndim == 2, 'x_train and y_train should be 2D arrays'
        assert x_train.shape[0] == y_train.shape[0], 'first dimension of x_train and y_train has to be equal (n_points)'
        assert y_train.shape[1] == 1, 'y_train has to be of form (n_points, 1)'
        self.x_train = x_train
        self.y_train = y_train
        # Due to numerical instabilities, the covariance matrix might not be invertible.
        # We add a small constant value to the diagonal elements ('jitter') to enfore the
        # matrix to be positive semidefinite (which implies invertibility)
        # See, e.g., https://scicomp.stackexchange.com/questions/36342/advantage-of-diagonal-jitter-for-numerical-stability
        #
        #
        # The inputs x_train and y_train both have to be 2D here. x_train has shape (n_points, dim_of_points), y_train has
        # shape (n_points, 1)
        #
        # ➡️ TODO : compute the covariance and the inverse of the covariance matrix here. save them as class attributes so we don't need to
        #  recompute them all the time ⬅️
        #
        self.cov = ...
        self.cov_inv = ...
        return self

    def __call__(self, x: np.ndarray) -> sp.stats._multivariate.multivariate_normal_frozen:
        assert self.x_train is not None and self.y_train is not None, "Have to initialize the GP before calling"
        #
        # ➡️ TODO : compute the posterior distribution ⬅️
        #
        # First, compute the posterior mean and covariance (use self.cov_inv and the definitions above), 
        # then compute the posterior distribution which is a multivariate normal distribution
        # Hint: you might need to add a small diagonal value to the covariance matrix again
        # Use the code from above for that (don't add more than 1e-6)

        if x.ndim == 1:
            x = x[np.newaxis, :]
            
        return ...

    def posterior_mean(self, x: np.ndarray) -> np.ndarray:
        if x.ndim == 1:
            x = x[np.newaxis, :]
        dist = self(x)
        return dist.mean

    def posterior_covariance(self, x: np.ndarray) -> np.ndarray:
        if x.ndim == 1:
            x = x[np.newaxis, :]
        dist = self(x)
        return dist.cov

    def log_marginal_likelihood(
            self,
    ) -> float:
        #
        # ➡️ TODO : Implement this when you're instructed to in the notebook, you can skip this for now otherwise ⬅️
        # 
        
        return ...

**Safety check:** Run the following cell to make sure that everything is working correctly.

In [ ]:
if isinstance(
        GaussianProcess(mean, kern, np.random.rand()).initialize(np.random.rand(5, 1), np.random.rand(5, 1))(
            np.random.rand(3, 1)
        ),
        sp.stats._multivariate.multivariate_normal_frozen
):
    print("✅ All good.")
else:
    print("🚨 __call__ does not return a multivariate distribution. Make sure to use 'sp.stats.multivariate_normal'")

## Initializing and fitting the Gaussian process

In [ ]:
#
# ➡️ TODO : Create a new GP instance with a lengthscale of your choice. 
# Try different lengthscales to see how the affect the behavior of the GP.  ⬅️
#       

gp = ...
gp.initialize(x_train, y_train)

### Plotting utility

In [ ]:
def plot_gp(gp: GaussianProcess, x_test: np.ndarray, y_test: np.ndarray, ax: plt.Axes = None) -> None:
    """
    Plot the GP posterior distribution of gp for a given set of test points (x_test and y_test).
    Accepts an optional parameter ax which plots it on an existing ax, otherwise a new figure
    is create.
    """
    if ax is None:
        fig, ax = plt.subplots(1, 1)
    posterior_distribution = gp(x_test)
    rvs = posterior_distribution.rvs(10)

    ci_lb = []
    ci_ub = []

    for _x in x_test:
        x_marginal = gp(_x)
        _mean = x_marginal.mean.squeeze()
        variance = x_marginal.cov.squeeze()

        _lb, _ub = sp.stats.norm.interval(0.95, loc=_mean, scale=np.sqrt(variance))
        ci_lb.append(_lb)
        ci_ub.append(_ub)

    ci_lb = np.array(ci_lb)
    ci_ub = np.array(ci_ub)

    for i, rv in enumerate(rvs):
        x_test_sq = x_test.squeeze()
        ax.plot(x_test.squeeze(), rv, alpha=0.25, color='blue', label='posterior sample' if i == 0 else None)
    ax.fill_between(x_test.squeeze(), ci_lb, ci_ub, color='gray', alpha=0.5, label='95% CI')
    ax.plot(x_test, y_test, color='red', label='y(x)')
    ax.scatter(x_train, y_train, marker='x', color='black', label='training data')
    ax.set_ylabel('y(x)')
    ax.legend()

## Plot the above GP for the test data defined earlier

In [ ]:
plot_gp(gp, x_test, y_test)
plt.show()

In the plot we see the predicted mean, the 95% error bars together with a few functions sampled from the Gaussian distribution.

# Task: play around with lengthscales

* What happens if too low?
* What happens if too high?

## Systematic approach to fit lengthscales: MLE

We saw above that setting the lengthscale incorrectly can render the Gaussian process useless: if it is set too low or too high, the Gaussian process fails in modelling the function reasonably (if you didn't see that, try lengthscales of 0.1 and 10).
While you probably found a lengthscale value that led to reasonable performance, we want to set the lengthscale automatically because what a "good value" is depends on the function at hand.
All approaches to set the lengthscale (and often more hyperparameters) are in some form **maximizing the marginal likelihood of seeing the training data under the GP prior** (i.e., we will optimize our lengthscale with MLE).


Note that $\mathbf{y}$ is a $n$-dimensional vector which represents one possible realization of the underlying true function at the locations $X \in \mathbb{R}^{n\times d}$.
The vector $\mathbf{y}|X$ follows a multivariate normal distribution, and as such:
$$
\hat{l} = \arg\max_l p(\mathbf{y}|X,l)
$$

We state the posterior (log) marginal likelihood here and refer to [Gaussian Processes for Machine Learning](https://gaussianprocess.org/gpml/chapters/RW.pdf) for a derivation:
$$
\log p(\mathbf{y}|X,l) = -\frac{1}{2}\mathbf{y}^\intercal K^{-1}\mathbf{y}-\frac{1}{2}\log |K| -\frac{n}{2}\log 2\pi.
$$

Again, we switch between the notation $K$ and $k(x_{1:n}, x_{1:n})$, but they mean the same thing.


**YOUR TASK**: Implement the `log_marginal_likelihood` in the `GP` class above.

In [ ]:
# We define a function that we can pass to the minimize function from scikit learn

def negative_marginal_log_likelihood(log_ls: float) -> float:
    ls = np.exp(log_ls)
    gp = GaussianProcess(mean, kern, ls)
    gp.initialize(x_train, y_train)
    return -gp.log_marginal_likelihood()

### Maximize the marginal likelihood by minimizing the negative marginal log likelihood

We optimize the negative_marginal_log_likelihood over the log of the lengthscale intead of directly over the parameter.

**Question:** What is the benefit of doing that?

**Safety check:** The following cell should print something close to `[0.76656]`

In [ ]:
best_log_ls = sp.optimize.minimize(negative_marginal_log_likelihood, 1)['x']
best_ls = np.exp(best_log_ls)
print(best_ls)

### Plot the GP with lengthscale fitted by MLE

In [ ]:
gp = GaussianProcess(mean, kern, best_ls)
gp.initialize(x_train, y_train)

posterior_distribution = gp(x_test)

plot_gp(gp, x_test, y_test)
plt.show()

## Towards Bayesian Optimization: Acquisition functions

We now have all the ingredients of the Gaussian process and focus on the actual task: finding the optimizer of a function (i.e., the point with the best function value).
The general strategy is to first fit a Gaussian process on a small number of initial training points and then find the next point to evaluate.

We find this next point by maximizing a so-called _acquisition function_.
While it may seem strange to solve an optimization problem (maximizing the acquisition function) to maximize our black-box function, the acquisition function can be maximized using gradient-based approaches because it has a closed-form expression.

We will work with a popular acquisition function: __Expected improvement (EI)__.
EI describes by how much, in expectation w.r.t. to our GP posterior after $n$ observations, a given point improves over current best function value observed so far $y^*_n$:
$$
EI_n(x) = \mathbb{E}_{y\sim GP(X,\mathbf{y})}\left[[y(x)-y^*_n]^+ \right]
$$
Note that $[x]^+:=\max(0,x)$.

EI naturally does an _exploration-exploitation tradeoff_: points that have already been evaluated get zero EI (in noiseless models as in this lab) but regions where all possible functions have a value worse than the current best point also get zero EI. 
The sweet spot are regions that are under-explored but are also promising.

Since our GP posterior follows a multivariate normal distribution, we can find a [closed form expression for the expected improvement](https://ekamperi.github.io/machine%20learning/2021/06/11/acquisition-functions.html). Let the mean and variance of our new point $(x,y)$ be $\mu(x)$ and $\sigma(x)$, as given by the GP. Then:
$$
EI_n(x) = \sigma(x)\varphi (Z(x)) + \sigma(x)Z(x)\Phi (Z(x)),
$$
where $Z(x) = \frac{\mu_n(x)-y^*_n}{\sigma}$ is the expected difference between the point $x$ and the best function values after $n$ observations. Further, $\varphi(x)$ is the probability density function of the *standard* normal distribution, and $\Phi$ is the cummulative density function of the *standard* normal distribution.

__YOUR TASK:__ Implement the expected improvement acquisition function by adding missing lines in the cell below. You can use sp.stats for pds and cdfs.

In [ ]:
def expected_improvement(
    gp: GaussianProcess,
    x: np.ndarray,
    best_observed_value: float
) -> np.ndarray:
    
    if x.ndim == 1:
        x = x[np.newaxis, :]
    eis = []
    for _x in x:
        # ➡️ TODO : Implement the expected improvement acquisition function by adding missing lines  ⬅️
        ...
        eis.append(_ei.squeeze())
    return np.array(eis)

__Safety check:__ (make sure you executed all cells in order up to here, i.e., you used the GP with MLE). 

You should get the following figure:

![EI plot](EI.png)

In [ ]:
plt.plot(expected_improvement(gp, np.linspace(lb, ub, 300).reshape(-1, 1), y_train.max()).squeeze(), label=rf'$EI_n(x)$')
plt.legend()
plt.show()

## Bayesian optimization

Now, we have all the ingredients to do Bayesian optimization. We will 
* Sample some initial points and evaluate them
* Fit a GP model with MLE
* Find the next point to evaluate by maximizing the Expected improvement
* Evaluate the next point, add it to the data and start over from the second point 

We will now see how all components play together.

### Defining a function to maximize the likelihood

We wrap our MLE procedure in a single function that we can call conveniently later on. Given `mean`, `kern`, `x_train`, and `y_train`, this function returns a GP where the lengthscale is set by MLE. 

**Question (after running the BO loop):** Why is the model of the function so accurate in the middle part of the function but poor in the outer parts?

In [ ]:
def maximize_likelihood(
        mean: Callable[[np.ndarray], np.ndarray],
        kern: Callable[[np.ndarray], np.ndarray],
        x_train: np.ndarray, y_train: np.ndarray
) -> GaussianProcess:
    gp = GaussianProcess(mean, kern, 0.1)
    gp.initialize(x_train, y_train)

    def likelihood_function(log_ls: float) -> float:
        gp = GaussianProcess(mean, kern, np.exp(log_ls))
        gp.initialize(x_train, y_train)
        return -gp.log_marginal_likelihood()

    best_log_ls = sp.optimize.minimize(likelihood_function, 1)['x']
    best_ls = np.exp(best_log_ls)
    
    gp = GaussianProcess(mean, kern, best_ls)
    gp.initialize(x_train, y_train)

    return gp

In [ ]:
x_train = np.random.RandomState(2).rand(5) * (ub - lb) + lb
x_train = x_train[:, np.newaxis]
y_train = fn(x_train)

gp = maximize_likelihood(mean, kern, x_train, y_train)
ei = lambda x: expected_improvement(gp, x, y_train.max())

for i in range(10):
    print('###########################')
    print(f"####### iteration {i} #######")
    print('###########################')

    fig, axs = plt.subplots(2, 1, sharex=True, figsize=(7, 7))
    # plot current GP
    plot_gp(gp, x_test, y_test, ax=axs[0])
    # stupidly optimize the acquisition function
    x_range = np.linspace(lb, ub, 1000)[:, np.newaxis]
    eis = ei(x_range)
    x_next = x_range[np.argmax(eis)]

    # plot acquisition function
    axs[1].plot(x_range, eis)
    axs[1].set_ylabel('EI(x)')
    axs[1].vlines(x_next, ymin=ei(x_next), ymax=0, color='green', linestyle='dashed', label='best x')
    axs[1].legend()
    plt.show(fig)
    # evaluate point where acquisition function maximum
    y_next = fn(x_next)
    # add to training data
    x_train = np.vstack((x_train, x_next[np.newaxis, :]))
    y_train = np.vstack((y_train, y_next[np.newaxis, :]))

    gp = maximize_likelihood(mean, kern, x_train, y_train)
    ei = lambda x: expected_improvement(gp, x, y_train.max())


***
## PART 2
***
# Bayesian optimization in action

In the second part of this lab, we will use Bayesian optimization to tune the hyperparameters of a neural network. Now when we have a little deeper understanding of how the Bayesian Optimization works internally, we will switch over and use a professional implementation, which is what one would use in practice. Here we will use the [BoTorch](https://botorch.org/) library developed at Meta, which is arguably the most efficient and well maintaned library at the moment. It uses the [Gpytorch](https://gpytorch.ai/) gaussian process library developed at Cornell University. 

We will use the white wine dataset, which contains ~5.000 different wines with physical attributes and a taste score from users that goes from 1 to 10. Our goal is to train a predictor to guess the quality of the wine from its physical attributes. The data comes from the paper

P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
Modeling wine preferences by data mining from physicochemical properties.
In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.

However, we only use a subset of the data set to speed up training.

The predictor will be a small (though probably too large) ANN, and the parameters that we will optimize now are hyperparameters of this ANN. Especially, we will optimize:
- Number of layers
- Number of neurons per layer
- Learning rate
- Dropout rate
- Number of training iterations

NOTE!!
The two parts of this lab are completely standalone, and as such, we will not re-use anything from Part 1 in Part 2.

![Wine plot](wine.jpg)

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import model_selection

torch.manual_seed(0);

In [ ]:
# Selecting the appropriate training device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device {device}")

### Load the dataset

First we load the data set and split it into train and test sets.
We select a small subset of the data for now, as it will be very slow to work with otherwise.

- Choose a number of data points to use.

In [ ]:
df = pd.read_csv("winequality-white.csv", delimiter=";")
print(df.head())
print(df.describe())

# to speed up training we only use a very small subset of the data set. Feel free to play around. Especially if you have GPU support.
# ➡️ TODO : select how many data samples to use in the lab. We recommend 50 or 100 for now.  ⬅️
n_data_samples = ...

X = df.values[:n_data_samples, :-1]
y = df.values[:n_data_samples, -1]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

# to keep this data separate from the X, y used in the bayesian optimization below, we use a D in the name 
DX_train = torch.tensor(X_train).to(dtype=torch.double, device=device)
DX_test = torch.tensor(X_test).to(dtype=torch.double, device=device)
Dy_train = torch.tensor(y_train).to(dtype=torch.double, device=device)
Dy_test = torch.tensor(y_test).to(dtype=torch.double, device=device)

We will use a relatively small ANN architecture with up to 4 hidden layers. We use dropout, with dropout probabilities that are subject to optimization. Furthermore, use ReLU activation functions.

In [ ]:
# Here we define the general architecture for the ANN. It takes 3 parameter: number of hidden layers, 
# number of neurons per hidden layer and dropout probability.

class ANN(torch.nn.Module):
    
    def __init__(
        self,
        n_hidden_layers: int,
        hidden_width: int,
        dropout_p: float
    ):
        super().__init__()
        self.model = torch.nn.Sequential(            
            torch.nn.Linear(torch.tensor(11), hidden_width),
            torch.nn.Dropout(dropout_p),
            torch.nn.ReLU(),
            *(
            torch.nn.Linear(hidden_width, hidden_width),                
            torch.nn.Dropout(dropout_p),
            torch.nn.ReLU(),
            ) * (n_hidden_layers),        
            torch.nn.Linear(hidden_width, 1)
        )

    def forward(self, x):
        return self.model(x)


We will use the Adam optimizer with a learning rate that is subject to optimization. We will use the Mean Squared Error loss function.

To make the BO cleaner, we define the full training procedure in a function `black_box_function` that takes a vector of hyperparameters as input and returns the accuracy on the test set as output. We will use this function as a black box in the optimization. 

Note!! The black_box_function takes inputs in the range [0,1] and then maps those to pre-set ranges inside the black_box_function.

In [ ]:
def black_box_function(x: torch.Tensor) -> float:
    """
    This function takes a (nx5) input tensor x, where each value is between 0 and 1 (inclusive).
    It then maps those 0-1 values into the actual values that the ANN needs.
    
    x[0]: number of hidden layers
    x[1]: number of neurons per layer
    x[2]: dropout p
    x[3]: learning rate
    x[4]: num training iterations
    """

    # Map the values
    n_hidden = int(torch.floor(x[0] * 3)) # maps from [0,1] -> {0,..,3}
    n_neurons = int(torch.floor(x[1] * 196)) + 5 # maps from [0,1] -> {5, 200}
    dropout_p = 0.1 * x[2] # maps from [0,1] -> [0, 0.1]
    learning_rate = 0.1 * x[3] # maps from [0,1] -> [0, 0.1]
    num_gd_iters = int(torch.floor(x[4] * 901) + 100) # maps from [0,1] -> {100, 1000}

    print(f'dropout: {dropout_p:.3E}', end='\t')
    print(f'lr: {learning_rate:.3E}', end='\t')
    print(f'depth: {n_hidden}', end='\t')
    print(f'width: {n_neurons}', end='\t')
    print(f'gd_iter: {num_gd_iters}')

    # create the model
    model = ANN(int(n_hidden), int(n_neurons), dropout_p).to(dtype=torch.double, device=device)

    # Train the model
    loss_fn = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate.item())    
    model.train()
    for epoch in range(num_gd_iters):
        y_pred = model(DX_train)
        loss = loss_fn(y_pred.reshape(-1), Dy_train.reshape(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()        
    model.eval()

    # calculate train and test loss
    with torch.no_grad():
        y_train_pred = model(DX_train)
        y_test_pred = model(DX_test)
        train_loss = loss_fn(y_train_pred.reshape(-1), Dy_train.reshape(-1))
        test_loss = loss_fn(y_test_pred.reshape(-1), Dy_test.reshape(-1))
        print(f"Test loss = {test_loss:.3f}", f"[Training loss = {train_loss:.3f}]")
        

    # We minimize the test loss
    return test_loss.detach().cpu()

### We test that the black box function works

In [ ]:
default_performance = black_box_function(torch.tensor([0, 0,  0, 0, 0], dtype=torch.double, device=device))
print("test loss of default hyperparameters:", default_performance.item())

### Defining the GP model

We now have a model to optimize.

**Question:** What are we optimizing?

**Question:** What variables are we optimizing over?

Next thing is to create the BO loop.

We model the GP with GPyTorch. GPs can also model noisy functions. We haven't talked about noise above but the definitions of a noisy GP are almost identical to the ones of noiseless GPs.

In [ ]:
import gpytorch
from gpytorch.likelihoods import GaussianLikelihood

from botorch.models import SingleTaskGP


def get_gp(x_train: torch.Tensor, y_train: torch.Tensor) -> SingleTaskGP:
    # ➡️ TODO : Create a new SingleTaskGP and return the model  ⬅️
    # See https://botorch.org/api/_modules/botorch/models/gp_regression.html#SingleTaskGP for hints
    # You can try to improve your GP by choosing another kernel, another length scale prior, or 
    # another prior for the likelihood. The default should be fine though.

    model = ...
    return model

### Cheap function to test if everything works

You can use the following test function to check if your BO loop runs as intended before running on the expensive HPO problem. On the Branin problem, you should reach a value of around 0.4 after approximately 200 function evaluations.

In [ ]:
from botorch.test_functions import Branin

cheap_function = Branin()

We will random sample d+1 initial points to use as initial training data for the GP before starting to generate new points with BO. 
As above, we use Expected Improvement as the acquisition function. We will optimize the acquisition function using the L-BFGS algorithm. The Bayesian optimization loop will run for 24 iterations (and might take some time).

Note!! In part 1, we tried to maximize a function. In part two we are instead minimizing. Also, be careful with normalization/standardization. We want to train our GP model on normalized/standardardized data, but when we want to run our black box function, that takes unnormalized data as input. For the ANN blackbox it matters less, as that has [0,1] bounds already, but for the cheap test function it is important.



In [ ]:
from botorch.optim import optimize_acqf
from botorch.optim import optimize_acqf_mixed
from botorch.acquisition import LogExpectedImprovement
from gpytorch import ExactMarginalLogLikelihood
from botorch.utils.transforms import normalize, unnormalize, standardize
from botorch.fit import fit_gpytorch_mll
import itertools
import random

import torch


# ➡️ TODO : Set this to 'cheap_function' or 'black_box_function' 
function_to_optimize = ...

dim = 2 if isinstance(function_to_optimize, Branin) else 5

if not isinstance(function_to_optimize, Branin):
    """
    Some of are parameters in the HPO task are discrete. And while the black box function is written as to accept continuous inputs,
    the performance will be better if we let the BO algorithm know that it should only try discrete values.

    We do this by replacing the normal acquisition function optimizer "optimize_acqf()" from the botorch library with "optimize_acqf_mixed",
    where mixed stands for mixed continuous and discrete inputs.

    This mixed acqf optimizer requries a list of all points it should try for the discrete parameters which we calculate
    below and call fixed_feature_list. It is just the cartesian product of all discrete values. Note that we still
    normalize everything to [0, 1].
    """
    options_n_hidden = np.arange(4) / 3
    options_n_neurons = (np.arange(5, 201) - 5) / 201
    options_gd_iters = (np.arange(100, 1001) - 100) / 901

    fixed_features_list = []
    # iterate over the cartesian product of feasible locations and add a dictionary entry for each
    for _n_hid, _n_neur, _n_gd_it in itertools.product(options_n_hidden, options_n_neurons, options_gd_iters):
        fixed_features_list.append({
            0: _n_hid,  # 0: index of 'n_hidden'
            1: _n_neur,  # 1: index of 'n_neurons'
            4: _n_gd_it  # 2: # 0: index of 'n_gd_iters'
        })
    n_fixed_features = len(fixed_features_list)  # this is a rather long list


# First we need some random samples to train our GP on
# We use dim+1 initial random samples
x_init = torch.rand(dim+1, dim, dtype=torch.float64)

# Next we need to evaluate the black box function on each of those values
y_init = torch.tensor([function_to_optimize(x) for x in x_init]).reshape(-1, 1)

x = x_init
y = y_init

# Set to 50 for cheap function and to 24 for the HPO problem.
N_BO_STEPS = 50 if isinstance(function_to_optimize, Branin) else 24
# Set to 1 for the HPO problem
PRINT_EVERY = 5 if isinstance(function_to_optimize, Branin) else 1

print('*** Starting Bayesian Optimization ***')
for gp_iter in range(N_BO_STEPS):
    if gp_iter % PRINT_EVERY == 0:
        print(f'** Iteration {gp_iter + 1}/{N_BO_STEPS} **')
    # We define the bounds for the optimization. We assume that all hyperparameters are between 0 and 1.
    bounds = torch.stack([torch.zeros(dim), torch.ones(dim)]) if not isinstance(function_to_optimize, Branin) else torch.stack([torch.tensor([-5, 0]), torch.tensor([10, 15])])

    # ➡️ TODO : Normalize the x values to be between 0 and 1. You may use the botorch transforms  
    # https://botorch.org/api/_modules/botorch/utils/transforms.html    
    x_norm = ...

    # ➡️ TODO : Standardize the y values to have mean zero and standard deviation one.  
    y_stand = ...

    # ➡️ TODO : Create a new GP with the normalized x and y values 
    gp = ...

    # Your GP model has an attribute `likelihood` which you can use to compute the marginal log likelihood.
    # This attribute gives the term p(y|f,X,l) in the marginal log likelihood which in our case
    # is a Gaussian (see https://docs.gpytorch.ai/en/stable/likelihoods.html#gaussianlikelihood )
    # ➡️ TODO : Define the marginal log likelihood of the model (see https://docs.gpytorch.ai/en/stable/marginal_log_likelihoods.html#exactmarginalloglikelihood ) 
    # ➡️ TODO : Train the model (see https://botorch.org/api/fit.html) 
    mll = ...

    # ➡️ TODO : fit the model by maximizing the marginal likelihood (see https://botorch.org/api/fit.html#botorch.fit.fit_gpytorch_mll)
    ...

    # ➡️ TODO : Define an acquisition function for your model. We'll use the Log Expected Improvement (see https://botorch.org/api/_modules/botorch/acquisition/analytic.html#ExpectedImprovement )  
    # Note that we are minimizing now, so pass `maximize=False` as an argument.
    ei = ...

    # optimizing the acquisition function
    # set up the optimizer (for HPO and toy problem respectively)
    if not isinstance(function_to_optimize, Branin):
        # For the HPO task, we need to pass the fixed_features_list to the optimizer
        # however, we shuffle our list of values for discrete parameters and only keep the first 100 to keep a reasonably fast optimization
        optimizer = optimize_acqf_mixed
        random.shuffle(fixed_features_list)
        fixed_features_list_subsample = fixed_features_list[:100]
        kwargs = {
            "fixed_features_list": fixed_features_list_subsample
        }
    else:
        optimizer = optimize_acqf
        kwargs = {}

    # call the optimizer
    x_next, acq_value = optimizer(
        ei,
        bounds=torch.stack([torch.zeros(dim), torch.ones(dim)]),
        q=1,
        num_restarts=3,
        raw_samples=1024,        
        **kwargs
    )

    # We unnormalize x_next to be in the original bounds
    x_next_unnorm = unnormalize(x_next, bounds).detach().cpu()

    # Lastly we evaluate your black-box function on the point suggested by the acquisition function
    y_next = function_to_optimize(x_next_unnorm.squeeze())

    if gp_iter % PRINT_EVERY == 0:
        print(f'New function value: {y_next.item():.4f}. Current best: {y.min().item():.4f}')
        print('\n')
    x = torch.cat([x, x_next_unnorm])
    y = torch.cat([y, y_next.reshape(-1, 1)])

print("Finished!")

x_bo = x
y_bo = y

### Comparing BO to random search

We compare the performance of the models found by BO to randomly searching for ANN hyperparameters. Normally, we see that BO outperforms random search. Sometimes more, sometimes less. How does it perform in this case?

__Your task:__ Randomly initialize 30 ANNs and evaluate their performance. The performances should be in the 1D tensor `y_rs`.

In [ ]:
# ➡️ TODO : Randomly initialize 30 ANNs and evaluate their performance.  ⬅️

x_rs = ...
y_rs = ...

## Plotting the evolution of performance

Assuming that we have sequential setting where we run one configuration at a time, we can plot the accuracy of best configuration seen so far against time. This way, the y value for the two methods state the performacen we would have stopped after x iterations.

In [ ]:
y_bo_min = np.minimum.accumulate(y_bo)
y_rs_min = np.minimum.accumulate(y_rs)

plt.plot(y_bo_min, label='Bayesian Optimization')
plt.plot(y_rs_min, label='Random Search')
plt.yscale('log')
plt.legend()
plt.show()

Now if you want to, you can reuse this BO loop for your other machine learning projects, just replace the black box function for whatever you want to optimize. :) 

### The last thing we want to do is reconnect to the application

Find the best x values for the BO and the RNN applications, respectively. Then run the `black_box_function` on both and look at the ANN hyperparameter values it prints.

In [ ]:
# ➡️ TODO : Randomly initialize 30 ANNs and evaluate their performance.  ⬅️
x_best_rs = ...
x_best_bo = ...

print("Best hyperparameter values found for random search")
black_box_function(x_best_rs)
print("Best hyperparameter values found for random search")
black_box_function(x_best_rs)
print("Best hyperparameter values found for BO")
black_box_function(x_best_bo)

The GP model has internal parameters that regulates the parameter importance (lower means more important). You can print them as follows:

In [ ]:
for hyperparameter, lengthscale in zip(
    [
        "number of hidden layers:",
        "number of neurons per layer:",
        "dropout:\t\t",
        "learning rate:\t\t",
        "num training iterations:",
    ],
    gp.covar_module.lengthscale.squeeze()
):
    print(hyperparameter, "\t", lengthscale)
 

**Question:** Which parameters seem to matter a lot?

**Question:** Which parameters seem to matter a little?